# Implementação e Treinamento de uma CNN

Neste notebook, abordaremos o carregamento de datasets e a implementação de uma CNN.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

### Carregamento e Pré-processamento do Dataset CIFAR-10

O dataset CIFAR-10 é composto por 60.000 imagens coloridas de 32x32 pixels, distribuídas em 10 classes. Aplicaremos uma sequência de transformações (`transforms.Compose`):

1.  **`transforms.ToTensor()`**: Converte as imagens para tensores do PyTorch.
2.  **`transforms.Normalize()`**: Normaliza os tensores, escalando os valores dos pixels para o intervalo [-1, 1]. A equação de normalização para um canal de um pixel $x$ é:

    $$x_{norm} = \frac{x - \mu}{\sigma}$$

    Onde $\mu$ é a média e $\sigma$ é o desvio padrão.

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

batch_size = 64

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Classes do CIFAR-10
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### Visualização de Amostras de Dados

É uma boa prática inspecionar visualmente algumas amostras do dataset para garantir que os dados foram carregados e processados corretamente. O código a seguir extrai um lote (*batch*) de imagens do `train_loader` e as exibe.

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # desnormalizar
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(train_loader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images[:4]))
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

### Definição da Arquitetura da CNN

A arquitetura da CNN será definida utilizando `nn.Sequential` para agrupar camadas de forma modular. Dividiremos o modelo em duas partes principais:

* **Extrator de Features (`features`)**: Um bloco sequencial contendo as camadas convolucionais (`Conv2d`), de ativação (`ReLU`) e de pooling (`MaxPool2d`), responsável por aprender e extrair características hierárquicas das imagens.
* **Classificador (`classifier`)**: Um segundo bloco sequencial, que recebe o mapa de features achatado (*flattened*) e utiliza camadas totalmente conectadas (`Linear`) para realizar a classificação final.

### Implementação

Defina um modelo contendo 2 blocos, cada um contendo uma camada convolucional, uma função de ativação e um pooling.

In [ ]:
class SequentialCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        
        # Bloco de extração de features
        self.features = nn.Sequential(
            # Seu código aqui
        )
        
        # Bloco classificador
        self.classifier = nn.Sequential(
            # Seu código aqui
        )

    def forward(self, x):
        # Passa pelo extrator de features
        
        # Achata a saída para o classificador
        
        # Passa pelo classificador
        
        return x

model = SequentialCNN()

x = torch.randn(1, 3, 32, 32)
out = model(x)
print(out.shape)

In [ ]:
model = SequentialCNN().to(device)
print(model)

### Definição da Função de Perda e do Otimizador

Para o treinamento, utilizaremos a função de perda de **Entropia Cruzada** (`nn.CrossEntropyLoss`), adequada para problemas de classificação multiclasse. Como otimizador, empregaremos o **Adam** (`optim.Adam`), um algoritmo de otimização adaptativo eficiente.

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### Treinamento do Modelo

O loop de treinamento itera sobre o dataset por um número definido de épocas. Em cada iteração, realiza o *forward pass*, calcula a perda, executa o *backward pass* para computar os gradientes (backpropagation) e utiliza o otimizador para atualizar os pesos da rede.

In [ ]:
num_epochs = 15

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    # --- Treinamento ---
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # --- Validação ---
    model.eval()  # Coloca o modelo em modo de avaliação
    running_loss_val = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            loss = loss_function(outputs, labels)
            running_loss_val += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss = running_loss_val / len(test_loader)
    val_accuracy = 100 * correct_val / total_val
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    
    print(f'Epoch [{epoch + 1}/{num_epochs}] -> Train Loss: {train_loss:.3f}, Train Acc: {train_accuracy:.2f}% | Val Loss: {val_loss:.3f}, Val Acc: {val_accuracy:.2f}%')

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), train_accuracies, label='Training Accuracy')
plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

### Avaliação do Modelo

Após o treinamento, avaliamos a performance do modelo no conjunto de teste. O cálculo de gradientes é desativado com `torch.no_grad()` para otimizar o processo, uma vez que não há necessidade de retropropagação durante a inferência. A acurácia é calculada comparando as previsões do modelo com os rótulos verdadeiros.

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f} %')

## Exercícios

### Exercício 1

O modelo está muito simples para o problema. Aumente o número de canais nas camadas convolucionais. Além disso, adicione Batch Normalization após as camadas convolucionais e Dropout após as camadas lineares no modelo. Compare com a implementação original.

### Exercício 2

Experimente diferentes combinações para o número de filtros nas duas camadas convolucionais. Avalie todas as combinações possíveis e identifique o modelo que obtiver o melhor desempenho no conjunto de validação.

### Exercício 3

Explore diferentes configurações escolhidas aleatoriamente para o número de filtros nas camadas convolucionais e dos neurônios nas camadas lineares. Treine e avalie N configurações, registrando o desempenho em validação. Identifique o modelo com os melhores resultados entre as amostras testadas.